In [ ]:
#SIMULATION - POLICY EFFECTIVENESS (Readme)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Module conducts the following analysis:

# ---- Figure 6: Differences in expected green H2 demand and policy cost effectiveness for a spatially targeted demand-side CfD
# ---- Supplementary Figure 9: Differences in expected green H2 demand and policy cost effectiveness for a spatially targeted demand-side CfD: Results for alternative baselines
# ---- Supplementary Figure 10: Expected green H2 demand and policy cost effectiveness for a spatially targeted demand-side CfD

# Module is input for:

# ---- Simulation-policy-effectiveness#
# ---- Simulation-treatment

In [ ]:
#SET-UP
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.getenv("PROJ_LIB")

check_and_load <- function(packages) {
  for (pkg in packages) {
    if (!requireNamespace(pkg, quietly = TRUE)) {
      message(paste("Installing missing package:", pkg))
      install.packages(pkg, dependencies = TRUE, repos = "https://cloud.r-project.org")
    }
    if (!(pkg %in% (.packages()))) {
      suppressPackageStartupMessages(library(pkg, character.only = TRUE))
    }
  }
}

# --- Required Libraries 
required_packages <- c(
  # Core data handling
  "dplyr",      # Data manipulation and summarisation
  "purrr",      # Functional programming tools (map, reduce, etc.)
  "data.table",
  "stringr",
   "tidyr",
  
  # Visualization
  "ggplot2",    # Core plotting
  "ggdist",     # Distributional visualisations (e.g., ridgelines, intervals)
  "patchwork"   # Combine multiple ggplots into a single layout
)

check_and_load(required_packages)

In [ ]:
#DATAFILES
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
results_all_dist <- readRDS("results_all_dist.rds")
results_all_dist_mean_restricted <- readRDS("results_all_dist_restricted_mean.rds")
results_all_dist_mean_extended <- readRDS("results_all_dist_extended_mean.rds")
results_all_dist_cons_central <- readRDS("results_all_dist_central_conservative.rds")
results_all_dist_cons_restricted <- readRDS("results_all_dist_restricted_conservative.rds")
results_all_dist_cons_extended <- readRDS("results_all_dist_extended_conservative.rds")
results_all_dist_progr_central <- readRDS("results_all_dist_central_progressive.rds")
results_all_dist_progr_restricted <- readRDS("results_all_dist_restricted_progressive.rds")
results_all_dist_progr_extended <- readRDS("results_all_dist_extended_progressive.rds")

In [ ]:
#CALCUALTE RELEVANT METRICS
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


# Metrics
all_metrics_raw <- map_dfr(names(results_all_dist), function(name) {
  map_dfr(seq_along(results_all_dist[[name]]), function(i) {
    results_all_dist[[name]][[i]] %>%
      arrange(year) %>%
      mutate(
        annual_demand     = annual_demand / 1e3,          # kt → Mt
        cumulative_demand = cumsum(annual_demand),
        cumulative_cost   = cumsum(annual_cost),
        simulation        = i,
        centrality_type   = case_when(
          grepl("betweenness", name) ~ "Betweenness",
          grepl("degree",      name) ~ "Degree"
        ),
        intervention = case_when(
          grepl("_high",     name) ~ "High Centrality",
          grepl("_low",      name) ~ "Low Centrality",
          grepl("_random",   name) ~ "Random",
          grepl("_baseline", name) ~ "Baseline"
        )
      )
  })
})

all_metrics_raw <- all_metrics_raw %>%
  mutate(centrality_type = factor(centrality_type,
                                  levels = c("Degree", "Betweenness")))

# Baseline-adjusted Metrics
baseline_all_years <- all_metrics_raw %>%
  filter(intervention == "Baseline") %>%
  select(simulation, centrality_type, year,
         cumulative_baseline_demand = cumulative_demand,
         annual_baseline_demand     = annual_demand,
         cumulative_baseline_cost   = cumulative_cost)

metrics_all_dist <- all_metrics_raw %>%
  filter(intervention != "Baseline") %>%
  left_join(baseline_all_years,
            by = c("simulation", "centrality_type", "year")) %>%
  mutate(
    cumulative_additional_demand = cumulative_demand - cumulative_baseline_demand,
    annual_additional_demand     = annual_demand     - annual_baseline_demand,
    efficiency = ifelse(cumulative_additional_demand > 0,
                       cumulative_cost / cumulative_additional_demand,
                        NA_real_)
  )

# Median and IQR
summ_by_year <- function(df, value) {
  df %>%
    group_by(year, centrality_type, intervention) %>%
    summarise(
      med = median({{ value }}, na.rm = TRUE),
      p25 = quantile({{ value }}, .25, na.rm = TRUE),
      p75 = quantile({{ value }}, .75, na.rm = TRUE),
      .groups = "drop"
    )
}

eff_time <- summ_by_year(metrics_all_dist, efficiency)
ann_time <- summ_by_year(metrics_all_dist, annual_demand)

In [ ]:
# FIGURE 2 Expected green H2 and cost effectiveness for demand-side policy interventions targeted on spatial spillover potential
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Theme
plot_theme <- theme_minimal(base_size = 20) +
  theme(
    panel.grid       = element_blank(),
    panel.border     = element_rect(color = "black", fill = NA),
    axis.line        = element_line(color = "black"),
    axis.text        = element_text(size = 20)
  )

# Extract for calculation of PAIRED differences

extract_runs <- function(results_all_dist) {

  nm  <- names(results_all_dist)
  out <- vector("list", length(results_all_dist))

  for (idx in seq_along(nm)) {

    name <- nm[[idx]]
    runs <- results_all_dist[[name]]

    dt_name <- rbindlist(
      lapply(seq_along(runs), function(i) {
        s <- runs[[i]]
        data.table(
          run_id        = i,
          year          = s$year,
          annual_demand = s$annual_demand / 1e3,  # kt -> Mt
          annual_cost   = s$annual_cost,
          scenario_raw  = name
        )
      })
    )

    out[[idx]] <- dt_name
  }

  DT <- rbindlist(out)

  # centrality type
  DT[, centrality_type := ifelse(grepl("betweenness", scenario_raw),
                                 "Betweenness", "Degree")]

  # intervention category
  DT[, intervention := fifelse(grepl("_high",   scenario_raw), "High Centrality",
                        fifelse(grepl("_low",   scenario_raw), "Low Centrality",
                        fifelse(grepl("_random",scenario_raw),"Random",
                                "Baseline")))]

  setkey(DT, run_id, centrality_type, intervention, year)

  # cumulative values
  DT[, cumulative_demand := cumsum(annual_demand),
     by = .(run_id, centrality_type, intervention)]
  DT[, cumulative_cost   := cumsum(annual_cost),
     by = .(run_id, centrality_type, intervention)]

  # baselines
  baseDT <- DT[intervention == "Baseline",
               .(run_id, centrality_type, year,
                 base_demand = cumulative_demand,
                 base_cost   = cumulative_cost)]

  M <- merge(
    DT[intervention != "Baseline"],
    baseDT,
    by = c("run_id", "centrality_type", "year"),
    all.x = TRUE
  )

  M[, cumulative_additional_demand := cumulative_demand - base_demand]
  M[, cumulative_additional_cost   := cumulative_cost   - base_cost]

  M[, efficiency := ifelse(cumulative_additional_demand > 0,
                           cumulative_additional_cost / cumulative_additional_demand,
                           NA_real_)]

  M[, .(run_id, year, centrality_type, intervention,
        annual_demand, efficiency)]
}

runs_long <- extract_runs(results_all_dist)

# Differences
compute_diffs <- function(dt) {

  dt <- as.data.table(dt)

  long <- melt(
    dt,
    id.vars       = c("run_id", "year", "centrality_type", "intervention"),
    measure.vars  = c("annual_demand", "efficiency"),
    variable.name = "metric",
    value.name    = "value"
  )

  H <- long[intervention == "High Centrality"]
  R <- long[intervention == "Random"]
  L <- long[intervention == "Low Centrality"]

  setkey(H, run_id, year, centrality_type, metric)
  setkey(R, run_id, year, centrality_type, metric)
  setkey(L, run_id, year, centrality_type, metric)

  diff_HR <- H[R, .(
    run_id, year, centrality_type, metric,
    contrast = "diff_HvsR",
    diff = value - i.value
  )]

  diff_HL <- H[L, .(
    run_id, year, centrality_type, metric,
    contrast = "diff_HvsL",
    diff = value - i.value
  )]

  rbindlist(list(diff_HR, diff_HL))
}

diff_all <- compute_diffs(runs_long)

# Summary
summary <- diff_all[, .(
  med_diff = median(diff, na.rm = TRUE),
  p10_diff = quantile(diff, 0.25, na.rm = TRUE, type = 1),
  p90_diff = quantile(diff, 0.75, na.rm = TRUE, type = 1)
), by = .(year, centrality_type, metric, contrast)]

summary$centrality_type <- factor(
  summary$centrality_type,
  levels = c("Degree", "Betweenness")
)

demand_diff <- summary[metric == "annual_demand"]
eff_diff    <- summary[metric == "efficiency"]

# Y-limits
demand_range <- range(
  c(demand_diff$p10_diff, demand_diff$p90_diff),
  na.rm = TRUE
)
demand_ylim <- c(-max(abs(demand_range)), max(abs(demand_range)))

eff_range <- range(
  c(eff_diff$p10_diff, eff_diff$p90_diff),
  na.rm = TRUE
)
eff_ylim <- c(-max(abs(eff_range)), max(abs(eff_range)))

# Plot function
make_single_panel <- function(df_sub, title, ylab, ylim_use) {

  # Facet order
  df_sub <- df_sub %>%
    mutate(
      contrast = factor(
        contrast,
        levels = c("diff_HvsR", "diff_HvsL")  
      )
    )

  ggplot(df_sub, aes(x = year, y = med_diff, color = contrast)) +

    geom_errorbar(
      aes(ymin = p10_diff, ymax = p90_diff),
      width     = 0,
      linewidth = 0.75,
      alpha     = 0.35
    ) +
    geom_line(
      linewidth = 1.15
    ) +
    geom_hline(
      yintercept = 0,
      linewidth  = 0.7,
      colour     = "black"
    ) +

    # colour mapping
    scale_color_manual(
      values = c(
        "diff_HvsR" = "#2C7FB8",
        "diff_HvsL" = "#4DBBD5FF"
      ),
      labels = c(
        "diff_HvsR" = "High vs Random",
        "diff_HvsL" = "High vs Low"
      ),
      name = NULL
    ) +

    scale_y_continuous(limits = ylim_use) +

    scale_x_continuous(
      breaks = seq(2020, max(df_sub$year), by = 20),
      expand = expansion(mult = c(0.01, 0.12))
    ) +

    facet_wrap(
      ~ contrast,
      nrow = 1,
      labeller = as_labeller(c(
        "diff_HvsL" = "High vs Low",
        "diff_HvsR" = "High vs Random"
      ))
    ) +

    labs(
      title = title,
      x     = "Year",
      y     = ylab
    ) +

    plot_theme +
    theme(
      legend.position  = "none",
      strip.text       = element_text(size = 20, face = "plain"),
      strip.background = element_blank(),
      plot.title       = element_text(size = 20, face = "bold")
    )
}

# Panels
p_a <- make_single_panel(
  demand_diff[centrality_type == "Degree"],
  "Δ Green H2 Demand\nCfD targeted to Local Spillover Potential",
  "Δ Mt",
  demand_ylim
)

p_b <- make_single_panel(
  demand_diff[centrality_type == "Betweenness"],
  "Δ Green H2 Demand\nCfD targeted to Spillover Corridors",
  "Δ Mt",
  demand_ylim
)

p_c <- make_single_panel(
  eff_diff[centrality_type == "Degree"],
  "Δ Policy Cost Effectiveness\nCfD targeted to Local Spillover Potential",
  "Δ EUR/kg",
  eff_ylim
)

p_d <- make_single_panel(
  eff_diff[centrality_type == "Betweenness"],
  "Δ Policy Cost Effectiveness\nCfD targeted to Spillover Corridors",
  "Δ EUR/kg",
  eff_ylim
)

# Print and save

options(repr.plot.width = 18, repr.plot.height = 10, repr.plot.res = 600)

figure6 <- (
  p_a + p_b
) /
(
  p_c + p_d
)

figure6 <- figure6 +
  plot_layout(guides = "collect") +
  plot_annotation(tag_levels = "a") &
  theme(
    legend.position      = "none"
  )

print(figure6)

ggsave(
  "figure6.pdf",
  figure6,
  device = cairo_pdf,
  width  = 18,
  height = 10,
  dpi    = 600
)


In [ ]:
#ADDITIONAL VALUES (For text and annotations)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
abs_diff_2100 <- summary[year == 2100, .(
  centrality_type,
  metric,
  contrast,
  absolute_diff = med_diff
)]

# median levels in 2100
runs2100 <- runs_long[year == 2100]

control_2100 <- runs2100 %>%
  filter(intervention %in% c("Random", "Low Centrality")) %>%
  group_by(centrality_type, intervention) %>%
  summarise(
    control_annual = median(annual_demand, na.rm = TRUE),
    control_eff    = median(efficiency,    na.rm = TRUE),
    .groups = "drop"
  )

# small helper lookups
ctrl_rand <- control_2100 %>%
  filter(intervention == "Random") %>%
  select(centrality_type, control_annual_R = control_annual,
         control_eff_R    = control_eff)

ctrl_low  <- control_2100 %>%
  filter(intervention == "Low Centrality") %>%
  select(centrality_type, control_annual_L = control_annual,
         control_eff_L    = control_eff)

ctrl_all <- ctrl_rand %>%
  full_join(ctrl_low, by = "centrality_type")

# % diffs
final_2100 <- abs_diff_2100 %>%
  left_join(ctrl_all, by = "centrality_type") %>%
  mutate(
    denom = dplyr::case_when(
      metric == "annual_demand" & contrast == "diff_HvsR" ~ control_annual_R,
      metric == "annual_demand" & contrast == "diff_HvsL" ~ control_annual_L,
      metric == "efficiency"    & contrast == "diff_HvsR" ~ control_eff_R,
      metric == "efficiency"    & contrast == "diff_HvsL" ~ control_eff_L,
      TRUE ~ NA_real_
    ),
    pct_diff = 100 * absolute_diff / denom
  ) %>%
  select(centrality_type, metric, contrast, absolute_diff, pct_diff)

cat("\n================ ABSOLUTE + % DIFFERENCES IN 2100 ================\n")
print(as.data.frame(final_2100), row.names = FALSE)

In [ ]:
# theme
plot_theme <- theme_minimal(base_size = 20) +
  theme(
    panel.grid   = element_blank(),
    panel.border = element_rect(color = "black", fill = NA),
    axis.line    = element_line(color = "black"),
    axis.text    = element_text(size = 18),
    strip.text   = element_text(size = 18, face = "bold")
  )

# colors
slope_colors <- c(
  "Conservative" = "#CB181D",  
  "Mean"         = "#969696",  
  "Progressive"  = "#4292C6"
)



# scenario list
scenario_list <- list(
  "Mean – Restricted"         = results_all_dist_mean_restricted,
  "Mean – Central"            = results_all_dist,
  "Mean – Extended"           = results_all_dist_mean_extended,

  "Conservative – Restricted" = results_all_dist_cons_restricted,
  "Conservative – Central"    = results_all_dist_cons_central,
  "Conservative – Extended"   = results_all_dist_cons_extended,

  "Progressive – Restricted"  = results_all_dist_progr_restricted,
  "Progressive – Central"     = results_all_dist_progr_central,
  "Progressive – Extended"    = results_all_dist_progr_extended
)

# Extract runs (iN DT for efficiency)
extract_runs <- function(results_all_dist) {

  nm  <- names(results_all_dist)
  out <- vector("list", length(results_all_dist))

  for (idx in seq_along(nm)) {

    name <- nm[[idx]]
    runs <- results_all_dist[[name]]

    dt_name <- rbindlist(
      lapply(seq_along(runs), function(i) {
        s <- runs[[i]]
        data.table(
          run_id        = i,
          year          = s$year,
          annual_demand = s$annual_demand / 1e3,
          annual_cost   = s$annual_cost,
          scenario_raw  = name
        )
      })
    )

    out[[idx]] <- dt_name
  }

  DT <- rbindlist(out)

  DT[, centrality_type :=
       ifelse(grepl("betweenness", scenario_raw),
              "Betweenness", "Degree")]

  DT[, intervention := fifelse(
    grepl("_high", scenario_raw),   "High Centrality",
    fifelse(
      grepl("_low", scenario_raw),  "Low Centrality",
      fifelse(
        grepl("_random", scenario_raw), "Random",
        "Baseline"
      )
    )
  )]

  setkey(DT, run_id, centrality_type, intervention, year)

  DT[, cumulative_demand := cumsum(annual_demand),
     by = .(run_id, centrality_type, intervention)]
  DT[, cumulative_cost := cumsum(annual_cost),
     by = .(run_id, centrality_type, intervention)]

  baseDT <- DT[intervention == "Baseline",
               .(run_id, centrality_type, year,
                 base_demand = cumulative_demand,
                 base_cost   = cumulative_cost)]

  M <- merge(
    DT[intervention != "Baseline"],
    baseDT,
    by = c("run_id","centrality_type","year"),
    all.x = TRUE
  )

  M[, cumulative_additional_demand := cumulative_demand - base_demand]
  M[, cumulative_additional_cost   := cumulative_cost   - base_cost]

  M[, efficiency := ifelse(
    cumulative_additional_demand > 0,
    cumulative_additional_cost / cumulative_additional_demand,
    NA_real_
  )]

  M[, .(
    run_id,
    year,
    centrality_type,
    intervention,
    annual_demand,
    efficiency
  )]
}

# Paired differences
compute_diffs <- function(dt) {

  dt <- as.data.table(dt)

  long <- melt(
    dt,
    id.vars       = c("run_id","year","centrality_type","intervention"),
    measure.vars  = c("annual_demand","efficiency"),
    variable.name = "metric",
    value.name    = "value"
  )

  H <- long[intervention == "High Centrality"]
  R <- long[intervention == "Random"]
  L <- long[intervention == "Low Centrality"]

  setkey(H, run_id, year, centrality_type, metric)
  setkey(R, run_id, year, centrality_type, metric)
  setkey(L, run_id, year, centrality_type, metric)

  diff_HR <- H[R, .(
    run_id, year, centrality_type, metric,
    contrast = "High vs Random",
    diff = value - i.value
  )]

  diff_HL <- H[L, .(
    run_id, year, centrality_type, metric,
    contrast = "High vs Low",
    diff = value - i.value
  )]

  rbindlist(list(diff_HR, diff_HL))
}

# All Scenarios
all_summary <- rbindlist(
  lapply(names(scenario_list), function(nm) {
    dt  <- extract_runs(scenario_list[[nm]])
    dif <- compute_diffs(dt)
    dif[, scenario := nm]
    dif
  })
)

all_summary <- all_summary %>%
  mutate(
    slope = case_when(
      grepl("Progressive", scenario)   ~ "Progressive",
      grepl("Conservative", scenario)  ~ "Conservative",
      TRUE                             ~ "Mean"
    ),
    group = case_when(
      grepl("Extended", scenario)    ~ "Extended",
      grepl("Restricted", scenario)  ~ "Restricted",
      TRUE                           ~ "Central"
    )
  )

all_summary$centrality_type <- factor(
  all_summary$centrality_type,
  levels = c("Degree","Betweenness")
)

all_summary$group <- factor(
  all_summary$group,
  levels = c("Restricted","Central","Extended")
)

# Median and IQR
summary_diff <- all_summary[
  , .(
    med_diff = median(diff, na.rm = TRUE),
    q25      = quantile(diff, 0.25, na.rm = TRUE),
    q75      = quantile(diff, 0.75, na.rm = TRUE)
  ),
  by = .(year, centrality_type, group, slope, metric, contrast)
]

# Plot function
make_panel <- function(df_sub, title, ylab) {

  ggplot(df_sub, aes(x = year, group = slope)) +
    geom_ribbon(
      aes(ymin = q25, ymax = q75, fill = slope),
      alpha = 0.15,
      color = NA
    ) +
    geom_line(
      aes(y = med_diff, color = slope),
      linewidth = 1.05
    ) +
    geom_hline(yintercept = 0, color = "black") +
    scale_color_manual(values = slope_colors, name = "Scenario type") +
    scale_fill_manual(values = slope_colors, guide = "none") +
    facet_grid(
      rows = vars(group),
      cols = vars(centrality_type, contrast),
      scales = "free_y"
    ) +
    labs(title = title, x = "Year", y = ylab) +
    plot_theme
}

# Panels
p_a <- make_panel(
  summary_diff[metric == "annual_demand"],
  "Δ Green H₂ Demand",
  "Δ Mt"
)

p_b <- make_panel(
  summary_diff[metric == "efficiency"],
  "Δ Policy Cost Effectiveness",
  "Δ EUR/kg"
)

options(repr.plot.width = 18, repr.plot.height = 23, repr.plot.res = 600)

supplementary_figure_9 <- p_a / p_b +
  plot_annotation(tag_levels = "a")

print(supplementary_figure_9)

ggsave(
  "supplementary-figure-9.pdf",
  supplementary_figure_9,
  device = cairo_pdf,
  width  = 18,
  height = 23,
  units  = "in",
  dpi    = 800
)


In [ ]:
library(ggplot2)
library(patchwork)
library(dplyr)

# theme
plot_theme <- theme_minimal(base_size = 20) +
  theme(
    panel.grid       = element_blank(),
    panel.border     = element_rect(color = "black", fill = NA),
    axis.line        = element_line(color = "black"),
    axis.text        = element_text(size = 20),
    strip.background = element_blank(),
    strip.text       = element_text(size = 20, face = "plain"),
    plot.title       = element_text(size = 20, face = "bold"),
    legend.position  = "bottom"
  )

# colors
int_cols <- c(
  "High Centrality" = "#2C7FB8",
  "Low Centrality"  = "#4DBBD5FF",
  "Random"          = "#7F7F7F"
)

# plotting function
plot_abs_metric <- function(df, title, ylab) {

  ggplot(df, aes(x = year)) +

    # random: ribbon + dashed median
    geom_ribbon(
      data = df %>% filter(intervention == "Random"),
      aes(
        ymin = p25,
        ymax = p75,
        fill = intervention
      ),
      alpha = 0.25,
      color = NA
    ) +
    geom_line(
      data = df %>% filter(intervention == "Random"),
      aes(y = med),
      color = int_cols["Random"],
      linewidth = 1.0,
      linetype  = "dashed"
    ) +

    # high & low: error bars + lines
    geom_errorbar(
      data = df %>% filter(intervention != "Random"),
      aes(
        ymin  = p25,
        ymax  = p75,
        color = intervention
      ),
      width     = 0,
      linewidth = 0.7,
      alpha     = 0.35
    ) +
    geom_line(
      data = df %>% filter(intervention != "Random"),
      aes(
        y     = med,
        color = intervention
      ),
      linewidth = 1.2
    ) +

    # scales
    scale_color_manual(values = int_cols, name = NULL) +
    scale_fill_manual(values = int_cols, name = NULL) +

    # facets
    facet_wrap(
      ~ centrality_type,
      nrow = 1,
      labeller = as_labeller(c(
        Degree      = "CfD targeted to Degree Centrality (H₂ valleys)",
        Betweenness = "CfD targeted to Betweenness Centrality (H₂ corridors)"
      ))
    ) +

    # axes
    scale_x_continuous(
      breaks = seq(2020, max(df$year), by = 20),
      expand = expansion(mult = c(0.01, 0.10))
    ) +

    # labels
    labs(
      title = title,
      x     = "Year",
      y     = ylab
    ) +

    plot_theme +

    # legend control
guides(
  color = guide_legend(
    ncol = 1,
    override.aes = list(
      linetype  = "solid",
      linewidth = 1.2
    )
  ),
  fill = guide_legend(
    ncol = 1,
    override.aes = list(
      alpha = 0.25
    )
  )
)

}

# demand panels
p_a <- plot_abs_metric(
  ann_time %>% filter(intervention != "Baseline"),
  "Green H₂ Demand",
  "Annual Demand (Mt)"
)

# efficiency panels
p_b <- plot_abs_metric(
  eff_time %>% filter(intervention != "Baseline"),
  "Policy Cost Effectiveness",
  "EUR kg⁻¹"
)

# print and save
options(repr.plot.width = 25, repr.plot.height = 15, repr.plot.res = 600)

supplementary_figure_10 <- p_a / p_b +
  plot_layout(guides = "collect") +
  plot_annotation(tag_levels = "a")

print(supplementary_figure_10)

ggsave(
  "supplementary-figure-10.pdf",
  supplementary_figure_10,
  device = cairo_pdf,
  width  = 25,
  height = 15,
  dpi    = 600
)
